In [1]:
import pandas as pd
import numpy as np

In [2]:
POWER = pd.read_csv('lv2-2408.csv')
POWER.shape

(2781, 10)

In [3]:
POWER.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2781 entries, 0 to 2780
Data columns (total 10 columns):
DATE       2781 non-null object
SEASON     2781 non-null object
WEEKDAY    2781 non-null object
FAC        2781 non-null object
F1         2781 non-null float64
F2         2781 non-null float64
F3         2781 non-null float64
F4         2781 non-null float64
F5         2781 non-null float64
F6         2781 non-null float64
dtypes: float64(6), object(4)
memory usage: 217.4+ KB


### Question 21

In [4]:
# 21-1
df1 = POWER.copy()
for i in range(1, 7):
    col_name = f'F{i}'
    out_name = f'OUT{i}'
    df1[out_name] = np.where(df1[col_name] > df1.groupby(['SEASON', 'FAC'])[col_name].transform(lambda x: x.quantile(0.5) + 2 * (x.quantile(0.75) - x.quantile(0.25))), 1, 0)
df1['OUT'] = np.where((df1[['OUT1', 'OUT2', 'OUT3', 'OUT4', 'OUT5', 'OUT6']].sum(axis=1) > 0), 1, 0)
df1

,DATE,SEASON,WEEKDAY,FAC,F1,F2,F3,F4,F5,F6,OUT1,OUT2,OUT3,OUT4,OUT5,OUT6,OUT
0,2020-08-31,summer,Mon,Northern,624.23,484.21,30.36,35.57,273.27,320.81,0,0,0,0,0,0,0
1,2020-08-31,summer,Mon,Western,1106.89,1024.33,25.17,3.81,72.00,21.53,0,0,0,0,1,0,1
2,2020-08-31,summer,Mon,Southern,576.66,578.55,62.73,49.80,111.57,64.78,0,0,0,0,0,0,0
3,2020-09-01,fall,Tue,Northern,624.23,507.42,30.36,35.69,273.27,317.19,0,0,0,0,0,0,0
4,2020-09-01,fall,Tue,Western,1106.89,1050.91,25.17,3.83,72.00,23.97,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2776,2023-03-16,spring,Thu,Western,1326.61,1242.03,45.47,40.79,31.53,59.12,0,0,0,0,0,0,0
2777,2023-03-16,spring,Thu,Southern,639.01,599.81,67.16,65.43,81.21,88.48,0,0,0,0,0,0,0
2778,2023-03-17,spring,Fri,Northern,683.49,439.09,26.23,30.37,124.40,141.16,0,0,0,0,0,0,0
2779,2023-03-17,spring,Fri,Western,1326.61,1295.95,45.47,40.66,31.53,62.85,0,0,0,0,0,0,0


In [5]:
# 21-2
df1_out = df1.groupby(['SEASON', 'FAC'])['OUT'].mean().reset_index()
df1_out = df1_out.sort_values(by='OUT', ascending=False)
df1_out

,SEASON,FAC,OUT
8,summer,Western,0.320652
3,spring,Northern,0.184080
2,fall,Western,0.110294
9,winter,Northern,0.088889
6,summer,Northern,0.086957
11,winter,Western,0.040741
10,winter,Southern,0.037037
7,summer,Southern,0.005435
0,fall,Northern,0.000000
1,fall,Southern,0.000000


In [6]:
# Answer 21
ans21f = df1_out['OUT'].max()
ans21 = round(ans21f, 4)

print(ans21f)
print(ans21)

0.32065217391304346
0.3207


### Question 22

In [7]:
# 22-1
df2 = POWER[POWER['FAC'] == 'Northern'].copy()
print(df2.shape)

(927, 10)


In [8]:
# 22-2
df2['TOTAL'] = df2[['F1', 'F2', 'F3', 'F4', 'F5', 'F6']].sum(axis=1)
df2.head()

,DATE,SEASON,WEEKDAY,FAC,F1,F2,F3,F4,F5,F6,TOTAL
0,2020-08-31,summer,Mon,Northern,624.23,484.21,30.36,35.57,273.27,320.81,1768.45
3,2020-09-01,fall,Tue,Northern,624.23,507.42,30.36,35.69,273.27,317.19,1788.16
6,2020-09-02,fall,Wed,Northern,624.23,492.69,30.36,35.65,273.27,316.69,1772.89
9,2020-09-03,fall,Thu,Northern,624.23,568.34,30.36,35.62,273.27,297.47,1829.29
12,2020-09-04,fall,Fri,Northern,628.58,597.08,30.36,35.72,273.27,290.23,1855.24


In [9]:
# 22-3
cri = pd.DataFrame({'z': [1.64, 1.96, 2.33, 2.58], 'cump': [0.95, 0.975, 0.99, 0.995]})
cri['conf_level'] = 1 - (1 - cri['cump']) * 2
cri_use = cri[(cri['conf_level'].round(2) == 0.90)]['z'][0]
print('Used Z value:',cri_use)

Used Z value: 1.64


In [10]:
confs = df2.groupby('SEASON').apply(lambda x: pd.Series({
    'LB': x['TOTAL'].mean() - cri_use * 220 / np.sqrt(len(x)),
    'UB': x['TOTAL'].mean() + cri_use * 220 / np.sqrt(len(x))
})).reset_index()
confs['len'] = confs['UB'] - confs['LB']
confs

,SEASON,LB,UB,len
0,fall,1673.808434,1717.561860,43.753427
1,spring,1650.560235,1701.457974,50.897740
2,summer,1984.912667,2038.109725,53.197058
3,winter,1499.088930,1543.004107,43.915177


In [11]:
# 22-4
print(confs['len'].max())

53.19705778068692


In [12]:
# Answer 22
ans22f = confs['len'].max()
ans22 = round(ans22f, 1)

print(ans22f)
print(ans22)

53.19705778068692
53.2


### Question 23

In [13]:
# 23-1
df3 = POWER.copy()
df3 = df3[(df3['F1']!= 0) & (df3['F2']!= 0) & (df3['F3']!= 0) & (df3['F4']!= 0) & (df3['F5']!= 0) & (df3['F6']!= 0)]
df3 = df3.sort_values(by=['DATE', 'FAC'])
print(df3.shape)

(2778, 10)


In [14]:
# 23-2
df3['F6_NEXT'] = df3.groupby('FAC')['F6'].shift(-1)
df3 = df3.dropna(subset=['F6_NEXT'])
df3

,DATE,SEASON,WEEKDAY,FAC,F1,F2,F3,F4,F5,F6,F6_NEXT
0,2020-08-31,summer,Mon,Northern,624.23,484.21,30.36,35.57,273.27,320.81,317.19
2,2020-08-31,summer,Mon,Southern,576.66,578.55,62.73,49.80,111.57,64.78,59.49
1,2020-08-31,summer,Mon,Western,1106.89,1024.33,25.17,3.81,72.00,21.53,23.97
3,2020-09-01,fall,Tue,Northern,624.23,507.42,30.36,35.69,273.27,317.19,316.69
5,2020-09-01,fall,Tue,Southern,576.66,562.79,62.73,52.76,111.57,59.49,47.53
...,...,...,...,...,...,...,...,...,...,...,...
2774,2023-03-15,spring,Wed,Southern,639.01,579.85,67.16,51.29,81.21,64.32,88.48
2773,2023-03-15,spring,Wed,Western,1326.61,1182.94,45.47,40.93,31.53,39.30,59.12
2775,2023-03-16,spring,Thu,Northern,683.49,408.49,21.76,30.53,124.40,138.34,141.16
2777,2023-03-16,spring,Thu,Southern,639.01,599.81,67.16,65.43,81.21,88.48,77.41


In [15]:
df3[df3['FAC'] == 'Northern'][['DATE','F6','F6_NEXT']]

,DATE,F6,F6_NEXT
0,2020-08-31,320.81,317.19
3,2020-09-01,317.19,316.69
6,2020-09-02,316.69,297.47
9,2020-09-03,297.47,290.23
12,2020-09-04,290.23,296.10
...,...,...,...
2763,2023-03-12,132.12,142.25
2766,2023-03-13,142.25,144.71
2769,2023-03-14,144.71,140.37
2772,2023-03-15,140.37,138.34


In [16]:
# 23-3
df3 = df3.sort_values(by=['DATE', 'FAC'])
TRAIN3 = df3[df3['DATE'] < '2022-09-01'].copy()
TEST3 = df3[df3['DATE'] >= '2022-09-01'].copy()
print(TRAIN3.shape, TEST3.shape)

(2184, 11) (591, 11)


In [17]:
# 23-4
from sklearn.linear_model import LinearRegression
X = TRAIN3[['F1', 'F2', 'F3', 'F4', 'F5', 'F6']]
y = TRAIN3['F6_NEXT']
mdl3 = LinearRegression()
mdl3.fit(X, y)
print(mdl3.coef_, mdl3.intercept_)

[ 0.02855111 -0.02790587 -0.01737612 -0.03407228  0.09552583  0.90716697] 0.5439969096241271


In [18]:
# 23-5
TEST3['PRED'] = mdl3.predict(TEST3[['F1', 'F2', 'F3', 'F4', 'F5', 'F6']])
MAPE = np.mean(np.abs((TEST3['PRED']-TEST3['F6_NEXT'])/TEST3['F6_NEXT']))*100
MAPE

8.639223447522767

In [19]:
# Answer 23
ans23f = MAPE
ans23 = round(ans23f, 3)

print(ans23f)
print(ans23)

8.639223447522767
8.639


### Summary

In [20]:
print("Q21 Answer:", ans21)
print("Q22 Answer:", ans22)
print("Q23 Answer:", ans23)

Q21 Answer: 0.3207
Q22 Answer: 53.2
Q23 Answer: 8.639
